<a href="https://colab.research.google.com/github/wardasidd/MyDataScienceProjects/blob/main/Big_Mart_Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bigmart-sales-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9961%2F14084%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240825%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240825T221045Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8e6b57002ceff3d1d659597e4f9f1d3ea9946d86a7e91507169845e07e0de6faa1f5808e7158dcb6cbb09eb8ba7967a9ca7720252224cf43de78911905cba4842780864c17cbd7ae1c2b4b3374509bc8d0059fbf6b2dc34482f739c9c3dcae3f3f8feeefc47b5324ee65aca0eb5168ed52608f73244949e1ae18838fd7532880b5ca6204dffd38d33d62eaa11d7b85f2414bfe53960c805db39317b85f59f5983f3ca9aba7ad7c4c2dac13df0a1f73ef4ea66022c5d53048890e1397dba80c8056d6240a7470e355b377c6fb4655df29fe9bf22c902afba22be39758e1a9706964771dd26da9fdc18da2e381b3583469508f85849fcfd5c051bda358ab4faaa9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


## Objective:
To build a predictive model that can find out the sales of each product at a particular store and then provide actionable recommendations to the BigMart sales team to understand the properties of products and stores which play a key role in increasing sales

In [ ]:
#Importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as ex
import plotly.graph_objects as gb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# Read train and test data
train_df=pd.read_csv('/kaggle/input/bigmart-sales-data/Train.csv')
test_df=pd.read_csv('/kaggle/input/bigmart-sales-data/Test.csv')

In [ ]:
#checking the first five rows of the dataset
train_df.head()

# Observation:
As Item_Identifier and Outlet_identifier both are just for identity pupose and do not have any contribution in predicting the target variable (Item_Outlet_Sales) so we will be dropping these columns


In [ ]:
#dropping the columns from both datasets
train_df=train_df.drop(["Outlet_Identifier","Item_Identifier"],axis=1) #axis=1 represents column
test_df=test_df.drop(["Outlet_Identifier","Item_Identifier"],axis=1)

In [ ]:
#To understand more about each data frame such as non-null values and Dtype
train_df.info()
test_df.info()

1. Train dataset has total **8523** entries whereas test dataset has **5681** entries.
2. In both dataset **Item_Weight and Outlet_Size** has missing valus as no. of non-null values are less than the total rows or entries.
3. There are in total five coloumns with DType:Objest and those are **Outlet_Size, Outlet_Location_Type,Outlet_Type,Item_Type and Item_Fat_Content**
indicating these are strings or categorial valriables.

# **DATA PREPROCESSING AND EXPLORATORY DATA ANALYSIS**

In [ ]:
#Starting with visualizing the categorial variables

fig, axes = plt.subplots(2, 2, figsize = (18, 12))

fig.suptitle('Bar plot for all categorical variables in the dataset')

sns.countplot(ax = axes[0, 0], x = 'Item_Fat_Content', data = train_df, color = 'green',
              order = train_df['Item_Fat_Content'].value_counts().index);

sns.countplot(ax = axes[0, 1], x = 'Outlet_Size', data = train_df, color = 'black',
              order = train_df['Outlet_Size'].value_counts().index);

sns.countplot(ax = axes[1, 0], x = 'Outlet_Location_Type', data = train_df, color = 'red',
              order = train_df['Outlet_Location_Type'].value_counts().index);

sns.countplot(ax = axes[1, 1], x = 'Outlet_Type', data = train_df, color = 'blue',
              order = train_df['Outlet_Type'].value_counts().index);

# Observation:

1. By visulaizing **"Item_Fat_Content"**, we can clearly see that the Low Fat is also written as LF and low fat,whereas Regular is also written as reg.So this issue needs to be fixed.
2. **Outlet_Size** is categorized into Medium, Small and High Whereas **Outlet_Location_Type** is Classified into Tier 3 , Tier 2 and Tier 1.
3. In the **column Outlet_Type**, the majority of the stores are of Supermarket Type 1 and we have a less and almost equal number of representatives in the other categories including Supermarket Type 2, Supermarket Type 3, and Grocery Store

In [ ]:
#Visulaizing Item_Type - Categorial variable.
fig = plt.figure(figsize = (18, 6))

sns.countplot(x = 'Item_Type', data = train_df, color = 'teal', order = train_df['Item_Type'].value_counts().index);

plt.xticks(rotation = 45);

# Observation:
By Visualizing **Item_Type**, we observe that majority of the items sold in these stores are Fruits and Vegetables, followed by Snack Foods and Household items.

In [ ]:
# Before moving forward, Let's fix the issue we encountered in Item_Fat_content.
#Don't forget to fix it in both train and test data set
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Low Fat' if x == 'low fat' or x == 'LF' else x)
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Regular' if x == 'reg' else x)
test_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Low Fat' if x == 'low fat' or x == 'LF' else x)
test_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].apply(lambda x: 'Regular' if x == 'reg' else x)

In [ ]:
#Now Visulaizing Numerical Variables
fig, axes = plt.subplots(1, 3, figsize = (20, 6))
fig.suptitle('Histogram for all numerical variables')
sns.histplot(x = 'Item_Weight', data = train_df, kde = True, ax = axes[0]);
sns.histplot(x = 'Item_Visibility', data = train_df, kde = True, ax = axes[1],color='purple');
sns.histplot(x = 'Item_MRP', data = train_df, kde = True, ax = axes[2],color='pink' );

# Observation:
1. The **'Item_Weight'** variable exhibits an approximate uniform distribution. When addressing the missing values in this column, it's essential to ensure that the imputation does not substantially alter the original distribution.

2. On the other hand, the **'Item_Visibility'** variable is characterized by a right-skewed distribution. This indicates that while most items have a lower percentage of display area, a few items significantly exceed the average, occupying a much larger display area.

3. Lastly, the **'Item_MRP'** variable appears to follow a roughly multi-modal normal distribution. This suggests that the maximum retail price of items might cluster around several different values, rather than centering around a single mean.

# Bivariate Analysis

In [ ]:
#Lets move ahead with visualizing relationship between different variables

#Visualizing Relationship between Outlet_Establishment_Year & Item_Outlet_Sales

fig = plt.figure(figsize = (16, 5))
sns.lineplot(x = 'Outlet_Establishment_Year', y = 'Item_Outlet_Sales', data = train_df, errorbar=None, estimator = 'mean');

# Observation:
It's observed that the average sales have remained relatively stable annually, indicating no discernible upward or downward trend over time. Consequently, the year variable may not serve as a reliable predictor for forecasting sales, a hypothesis that can be further examined during the modeling stage.

Additionally, there's a notable dip in average sales in 1998. This sudden decrease might be attributed to external influences not captured within the dataset.

In [ ]:
#To visulaize relationship between every variable
numeric_df = train_df.select_dtypes(include=[np.number])
# Calculate the correlation matrix
corr = numeric_df.corr()
# Set up the matplotlib figure
fig = plt.figure(figsize=(18, 6))
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True)
# Rotate the x-axis labels
plt.xticks(rotation=45)
# Display the plot
plt.show()

# Observation:

Based on the visualization provided, it appears that **'Item_MRP'** (Maximum Retail Price) is the sole independent variable displaying a moderate linear association with the dependent variable, **'Item_Outlet_Sales'**. This suggests a potential trend where changes in 'Item_MRP' might correspond to changes in 'Item_Outlet_Sales', although the relationship is not particularly robust.

As for the other variables, they don't exhibit any marked positive or negative correlations with the dependent variable. This indicates that they may not significantly influence or predict the behavior of 'Item_Outlet_Sales' when considering linear relationships alone. Further analysis might be required to uncover more complex or subtle relationships within the data

In [ ]:
#scatter-plot of all Numercical variables with dependent variable
fig, axes = plt.subplots(1, 3, figsize = (20, 6))
fig.suptitle('Bi-variate scatterplot for all numerical variables with the dependent variable')
sns.scatterplot(x = 'Item_Weight', y = 'Item_Outlet_Sales', data = train_df, ax = axes[0]);
sns.scatterplot(x = 'Item_Visibility', y = 'Item_Outlet_Sales', data = train_df, ax = axes[1]);
sns.scatterplot(x = 'Item_MRP', y = 'Item_Outlet_Sales', data = train_df, ax = axes[2]);

# Observation:

The initial scatter plot illustrates a lack of discernible relationship between **'Item_Weight'** and **'Item_Outlet_Sales'**, indicating the data points are scattered randomly. This observation aligns with the previously noted weak correlation between these two variables, suggesting that **'Item_Weight'** might not be a significant predictor of **'Item_Outlet_Sales'.**

In the second scatter plot, examining **'Item_Visibility'** against **'Item_Outlet_Sales'**, no strong linear relationship is evident. However, a trend is observed where sales tend to decrease as **'Item_Visibility'** exceeds 0.19. This pattern might imply that items not typically sold frequently are allocated more visibility, possibly under the assumption that increased visibility would boost sales. This insight could lead to the creation of new features in the future, such as categorizing items into 'high visibility' and 'low visibility' groups, although this isn't pursued in the current analysis.

The third scatter plot, featuring **'Item_MRP'** versus **'Item_Outlet_Sales'**, distinctly shows a positive correlation. This indicates that as **'Item_MRP'** increases, **'Item_Outlet_Sales'** tend to increase as well, signifying that **'Item_MRP'** could be a strong predictor for sales and an important variable for future modeling efforts.

## Missing Value Treatment

In [ ]:
#Check for missing values
train_df.isnull().sum()

From the above result we can observe that **Item_Weight** and **Outlet_size** have 1463 and 2410 missing values respectively

Lets treat **Item_Weight** first:

In [ ]:
#Visulaizing each Item type's average Fat content

fig = plt.figure(figsize = (18, 3))
sns.heatmap(train_df.pivot_table(index = 'Item_Fat_Content', columns = 'Item_Type', values = 'Item_Weight'), annot = True);
plt.xticks(rotation = 45);

# Observation:
In the above heatmap,Different combinations of **Item_Types** and **Item_Fat_Content**, the average range of values for the column **Item_Weight** lies between the minimum value of 10 and the maximum value of 14.

In [ ]:
#Visulaizing Item_Weight with respect to Outlet_type and Location
fig = plt.figure(figsize = (18, 3))
sns.heatmap(train_df.pivot_table(index = 'Outlet_Type', columns = 'Outlet_Location_Type', values = 'Item_Weight'), annot = True);

As observed the average value for each location and Outlet_Type is 13

In [ ]:
# Item_weight with respect to Outlet_size
fig = plt.figure(figsize = (18, 3))
sns.heatmap(train_df.pivot_table(index = 'Item_Fat_Content', columns = 'Outlet_Size', values = 'Item_Weight'), annot = True);

It is also constant at 13 for every category.

In [ ]:
#Imputing Missing values in Item_Weight
#Randomnly Imputing from a uniform distribution over the interval [10, 14).

#For train dataset
item_weight_updated = train_df[train_df['Item_Weight'].isnull()].index
train_df.loc[item_weight_updated, 'Item_Weight'] = np.random.uniform(10, 14, len(item_weight_updated));

#For test dataset
item_weight_updated = test_df[test_df['Item_Weight'].isnull()].index
test_df.loc[item_weight_updated, 'Item_Weight'] = np.random.uniform(10, 14, len(item_weight_updated));


In [ ]:
outlet_size_data = train_df[train_df['Outlet_Size'].notnull()]
outlet_size_missing_data = train_df[train_df['Outlet_Size'].isnull()]
fig, axes = plt.subplots(1, 3, figsize = (18, 6))
fig.suptitle('Bar plot for all categorical variables in the dataset where the variable Outlet_Size is missing')
sns.countplot(ax = axes[0], x = 'Outlet_Type', data = outlet_size_missing_data, color = 'teal',
              order = outlet_size_missing_data['Outlet_Type'].value_counts().index);
sns.countplot(ax = axes[1], x = 'Outlet_Location_Type', data = outlet_size_missing_data, color = 'blue',
              order = outlet_size_missing_data['Outlet_Location_Type'].value_counts().index)
sns.countplot(ax = axes[2], x = 'Item_Fat_Content', data = outlet_size_missing_data, color = 'brown',
              order = outlet_size_missing_data['Item_Fat_Content'].value_counts().index);

# Observation:

Wherever Outlet_Size is missing in the dataset, most of the values are **Outlet_Type** as Supermarket Type 1, **Outlet_Location_Type** as Tier 2, and **Item_Fat_Content** as Low Fat.

In [ ]:
#understanding and visualizing non-null values in Outlet_size w.r.t Outlet_type
fig= plt.figure(figsize = (18, 3))
sns.heatmap(pd.crosstab(index = outlet_size_data['Outlet_Type'], columns = outlet_size_data['Outlet_Size']), annot = True,fmt='g')

# Observation:

From the heatmap analysis, it's apparent that grocery stores uniformly correspond to a 'Small' **Outlet_Size**.

Additionally, it's consistently observed that both Supermarket Type 2 and Supermarket Type 3 are categorized under a 'Medium' outlet size.

In [ ]:
#Visualizing Outlet_size w.r.t Outlet_Location_Type
fig = plt.figure(figsize = (18, 3))
sns.heatmap(pd.crosstab(index = outlet_size_data['Outlet_Location_Type'], columns = outlet_size_data['Outlet_Size']), annot = True, fmt = 'g')

# Observation:

As observed all the Tier2 locations are "small".

In [ ]:
#Visualizing Outlet_Size w.r.t Item_Fat_Content
fig = plt.figure(figsize = (18, 3))
sns.heatmap(pd.crosstab(index = outlet_size_data['Item_Fat_Content'], columns = outlet_size_data['Outlet_Size']), annot = True, fmt = 'g')

# Observation:

From the heatmap provided, There's no evident pattern among **'Item_Fat_Content'** and **'Outlet_Size'**.

In [ ]:
#Imputing Outlet_size='Small' where Outlet_Type='Grocery store' and Outlet_Location_Type ='Tier 2'
grocery_store= train_df[train_df['Outlet_Size'].isnull()].query(" Outlet_Type == 'Grocery Store' ").index
tier_2= train_df[train_df['Outlet_Size'].isnull()].query(" Outlet_Location_Type == 'Tier 2' ").index
train_df.loc[grocery_store, 'Outlet_Size'] = 'Small'
train_df.loc[tier_2, 'Outlet_Size'] = 'Small'

#Performing same on Test data
grocery_store= test_df[test_df['Outlet_Size'].isnull()].query(" Outlet_Type == 'Grocery Store' ").index
tier_2= test_df[test_df['Outlet_Size'].isnull()].query(" Outlet_Location_Type == 'Tier 2' ").index
test_df.loc[grocery_store, 'Outlet_Size'] = 'Small'
test_df.loc[tier_2, 'Outlet_Size'] = 'Small'

In [ ]:
#Check for null values again in train and test datasets
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

# Feature Engineering:

Having completed the steps of data understanding and preparation, we're now poised to advance to the modeling phase. At this juncture, it's recognized that while certain predictive features aren't directly present in the dataset, they could be ingeniously derived from the existing columns. This process of innovating new predictors from existing data attributes is referred to as **Feature Engineering**.

Embarking on this path, we hypothesize that a store's age might influence its sales, speculating that older stores could potentially have higher sales. To operationalize 'age', we'll calculate **'Outlet_Age'** by deducting the establishment year from 2013, the year when this data was compiled. This newly crafted feature, 'Outlet_Age', will thus reflect the respective ages of the outlets, and is introduced in the dataset through the following code.

In [ ]:
#Visulaizing the Hypothesis by plotting Outlet Sales by Outlet age
train_df['Outlet_Age'] = 2013 - train_df['Outlet_Establishment_Year']
test_df['Outlet_Age'] = 2013 - test_df['Outlet_Establishment_Year']

average_sales_by_age = train_df.groupby('Outlet_Age')['Item_Outlet_Sales'].mean()

plt.figure(figsize=(18, 6))
average_sales_by_age.plot(kind='line')
plt.title('Average Item Outlet Sales by Outlet Age')
plt.xlabel('Outlet Age')
plt.ylabel('Average Item Outlet Sales')
plt.grid(True)
plt.show()

# Observation:
Based on the visualizations produced, our initial hypothesis — that older stores would generally report higher sales — doesn't appear to be supported. The sales distributions across stores of varying ages seem to be roughly similar, indicating that store age may not be as significant a predictor of sales as initially presumed. However, we'll retain the 'Outlet_Age' feature for the time being. Its true impact and relevance will be more thoroughly assessed during the model-building phase. At that point, we can make a more informed decision about whether to keep or discard this variable, depending on its statistical significance and contribution to the model's performance.

## MODEL

In [ ]:
# Removing the target variable from the feature set
# Removing 'Outlet_Establishment_Year', as we have created a new variable 'Outlet_Age'
X_train = train_df.drop(['Item_Outlet_Sales', 'Outlet_Establishment_Year'], axis = 1)
# And then we are extracting the outcome variable separately
Y_train = train_df['Item_Outlet_Sales']

In [ ]:
# Creating dummy variables for the categorical variables
X_train = pd.get_dummies(X_train, drop_first = True)
X_train.head()

In [ ]:
# Creating an instance of the MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Applying fit_transform on the training features data
X_train_scaled = scaler.fit_transform(X_train)
# The above scaler returns the data in array format, below we are converting it back to pandas DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, index = X_train.index, columns = X_train.columns)
X_train_scaled.head()

In [ ]:
import statsmodels.api as sm
# Adding the intercept term
X_train_scaled = sm.add_constant(X_train_scaled)
# Calling the OLS algorithm on the train features and the target variable
ols_model_1 = sm.OLS(Y_train, X_train_scaled)
# Fitting the Model
ols_1 = ols_model_1.fit()
#Printing the summary of the model
print(ols_1.summary())

# Observation:

The R-squared value for our model stands at 0.563, indicating the proportion of variance in the dependent variable that's predictable from the independent variables. However, not all predictors are statistically significant in forecasting the outcome variable. To discern which variables hold predictive power, we need to examine the p-values associated with each independent variable.

# Interpreting the Regression Results:

**Adj. R-squared:** This metric adjusts the R-squared value based on the number of predictors and the sample size, providing a more accurate measure of model fit. Values range from 0 to 1, where higher values typically denote a better model fit. Here, the Adjusted R-squared is 0.562, suggesting that around 56.2% of the variability in the dependent variable is explained by the model.

**Coefficients:** These values indicate the expected change in the dependent variable for a one-unit change in the predictor, holding all other predictors constant.

**Std Err:** This represents the standard error of the estimated coefficients. Lower values suggest higher precision of the coefficient estimate.

**P >|t| (Pr(>|t|)):** The p-value tests the null hypothesis that the coefficient is equal to zero (no effect). A low p-value (< 0.05) indicates you can reject the null hypothesis. In other words, a predictor with a low p-value is considered to be statistically significant.

**Confidence Interval:** This range captures the true coefficient value with a certain probability (typically 95%). A narrow interval indicates a more precise estimate.


# Hypothesis Testing Framework:
**Null Hypothesis (H0):** It posits that there is no effect or no relationship between the variables. For instance, it might assert that the mean sales for different 'Outlet_Size' categories (High, Medium, Small) are equal.

**Alternative Hypothesis (Ha):** This is what you want to prove, suggesting that there is an effect or a relationship. For 'Outlet_Size' and 'Item_Outlet_Sales', it might claim that the mean sales for different outlet sizes are not equal.

If the p-value for a predictor is less than the significance level (often 0.05), we reject the null hypothesis in favor of the alternative. This means we have sufficient evidence to say there is a significant relationship between the predictor and the outcome variable.

# Observations from Model Summary:
Upon examining the model summary, only certain variables or specific categories of categorical variables might have a p-value lower than 0.05. These are the predictors that appear to have a statistically significant relationship with 'Item_Outlet_Sales'. It's these variables that we might consider focusing on in our model, as they have a more substantiated impact on predicting the outcome

# Feature Engineering

**Remove Multicollinearity**

- Muilticollinearity occurs when two or more predictor variables (also known as independent variables) in a regression model are highly correlated, meaning they have a strong linear relationship with each other.

There are different ways of detecting (or testing) multicollinearity. One such way is the Variation Inflation Factor.

- Variance Inflation factor: Variance inflation factor measures the inflation in the variances of the regression parameter estimates due to collinearities that exist among the predictors. It is a measure of how much the variance of the estimated regression coefficient βk is “inflated” by the existence of correlation among the predictor variables in the model.

- General Rule of thumb: If VIF is 1, then there is no correlation between the kth predictor and the remaining predictor variables, and hence the variance of β̂k is not inflated at all. Whereas, if VIF exceeds 5 or is close to exceeding 5, we say there is moderate VIF and if it is 10 or exceeds 10, it shows signs of high multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled.values, i) for i in range(X_train_scaled.shape[1])],
    index = X_train_scaled.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

Outlet_Age has a high VIF score. Hence, we are dropping Outlet_Age and building the model.

In [ ]:
X_train_scaled_new = X_train_scaled.drop("Outlet_Age", axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new.values, i) for i in range(X_train_scaled_new.shape[1])],
    index = X_train_scaled_new.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Buildiing the model and observing the p-values again
ols_model_2 = sm.OLS(Y_train, X_train_scaled_new)

ols_res_2 = ols_model_2.fit()

In [ ]:
print(ols_res_2.summary())

**OBSERVATION:**
When dealing with dummy variables, particularly those representing categories, it's generally not advisable to rely on Variance Inflation Factor (VIF) values to assess multicollinearity. This is because dummy variables are inherently correlated with their counterpart categories, often leading to naturally high VIF scores. Instead, a more effective approach is to examine the significance of these variables in the model, typically through their p-values.

Upon building the model and analyzing the p-values, it's observed that all the categories within the 'Item_Type' column have p-values exceeding the 0.05 threshold. This suggests that they do not significantly contribute to the model's predictive power. Consequently, it's reasonable to consider removing the entire 'Item_Type' column from the model, as it appears to offer little in the way of explanatory value.

In [ ]:
X_train_scaled_new2 = X_train_scaled_new.drop(['Item_Type_Breads',
'Item_Type_Breakfast',
'Item_Type_Canned',
'Item_Type_Dairy',
'Item_Type_Frozen Foods',
'Item_Type_Fruits and Vegetables',
'Item_Type_Hard Drinks',
'Item_Type_Health and Hygiene',
'Item_Type_Household',
'Item_Type_Meat',
'Item_Type_Others',
'Item_Type_Seafood',
'Item_Type_Snack Foods',
'Item_Type_Soft Drinks',
'Item_Type_Starchy Foods'], axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new2.values, i) for i in range(X_train_scaled_new2.shape[1])],
    index = X_train_scaled_new2.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Building the Model and obsering the p-values again
ols_model_3 = sm.OLS(Y_train, X_train_scaled_new2)

ols_res_3 = ols_model_3.fit()

print(ols_res_3.summary())

**OBSERVATION:**  Upon examining the p-values from the model, it's evident that the 'Item_Weight' column can be eliminated. This column possesses the highest p-value, indicating it is the least significant variable in terms of contributing to the model's predictive capability. Therefore, removing the 'Item_Weight' column is a justified step in refining the model and focusing on more impactful predictors.

In [ ]:
X_train_scaled_new3 = X_train_scaled_new2.drop("Item_Weight", axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new3.values, i) for i in range(X_train_scaled_new3.shape[1])],
    index = X_train_scaled_new3.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Building the model and observing p-values again
ols_model_4 = sm.OLS(Y_train, X_train_scaled_new3)

ols_res_4 = ols_model_4.fit()

print(ols_res_4.summary())

**OBSERVATION:**Based on the p-values observed, it's clear that both categories within the 'Outlet_Location_Type' column exceed the 0.05 threshold. This indicates that they do not significantly contribute to the model's performance. Consequently, it's reasonable to consider omitting the categories of 'Outlet_Location_Type' from the model due to their lack of statistical significance.

In [ ]:
X_train_scaled_new4 = X_train_scaled_new3.drop(["Outlet_Location_Type_Tier 2", "Outlet_Location_Type_Tier 3"], axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new4.values, i) for i in range(X_train_scaled_new4.shape[1])],
    index = X_train_scaled_new4.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Building the model and checking the p-values
ols_model_5 = sm.OLS(Y_train, X_train_scaled_new4)

ols_res_5 = ols_model_5.fit()

print(ols_res_5.summary())

From the above p-values, both the categories of the column Outlet_Size have a p-value higher than 0.05. So, we can remove the categories of Outlet_Size.

In [ ]:
X_train_scaled_new5 = X_train_scaled_new4.drop( ["Outlet_Size_Small", "Outlet_Size_Medium"], axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new4.values, i) for i in range(X_train_scaled_new4.shape[1])],
    index = X_train_scaled_new4.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Building the model and checking the p-values
ols_model_6 = sm.OLS(Y_train, X_train_scaled_new5)

ols_res_6 = ols_model_6.fit()

print(ols_res_6.summary())

Finally, lets drop Item_visibility.

In [ ]:
X_train_scaled_new6 = X_train_scaled_new5.drop( "Item_Visibility", axis = 1)

vif_series = pd.Series(
    [variance_inflation_factor(X_train_scaled_new5.values, i) for i in range(X_train_scaled_new5.shape[1])],
    index = X_train_scaled_new5.columns,
    dtype = float)

print("VIF Scores: \n\n{}\n".format(vif_series))

In [ ]:
#Building the model and checking the p-values
ols_model_7 = sm.OLS(Y_train, X_train_scaled_new6)

ols_res_7 = ols_model_7.fit()

print(ols_res_7.summary())

# OBSERVATIONS:
- All Variance Inflation Factor (VIF) scores are now below 5, indicating the absence of multicollinearity among predictors.
- The p-values for all remaining variables are below 0.05, signifying their statistical significance in the model.
- The R-Squared value remains approximately 0.56, suggesting that the eliminated variables were not contributing significantly to the model's explanatory power.

# Next Steps:
We'll proceed to verify the linear regression model's assumptions. If any issues are detected, we'll address them and consider remodeling. Specifically, we'll examine:

- The residuals' mean should be zero.
- The error terms should be normally distributed.
- There should be a linear relationship between predictors and the response variable.
- The presence of constant variance in the residuals (homoscedasticity).

In [ ]:
#Checking residual mean
residual = ols_res_7.resid
residual.mean()

It is almost equal to zero.

# Evaluating Normality in Residuals:

**Understanding the Test:**
The assumption of normally distributed error terms or residuals is fundamental in linear regression analysis. This normality ensures stable confidence intervals and reliable coefficient estimates. If residuals deviate from a normal distribution, it can lead to inaccuracies in these estimates and the overall model interpretation.

**Implications of Non-Normal Residuals:**

When residuals are not normally distributed, it might indicate outliers or anomalies in the data. These can significantly impact the model's performance and the reliability of predictions. It's essential to identify and understand these anomalies to improve model accuracy.

**Methods to Assess Normality:**

- Visual Inspection: Plotting a histogram of the residuals can provide a quick and intuitive understanding of their distribution. A symmetric, bell-shaped distribution suggests normality.
- Q-Q Plot: A Quantile-Quantile plot compares the distribution of residuals to a normal distribution. In a well-fitting model, the data points should approximately follow a straight line.
- Shapiro-Wilk Test: This statistical test specifically checks the hypothesis of normality. A significant p-value (typically less than 0.05) would indicate the residuals are not normally distributed.

**Remedial Measures for Non-Normal Residuals:**

If the residuals are found to be non-normal, various transformations can be considered to normalize them. Common transformations include logarithmic, exponential, and arcsine transformations. The choice of transformation depends on the specific pattern of non-normality and the underlying data distribution. Applying these transformations can help stabilize variance and make the model more robust and reliable.








In [ ]:
# Plot histogram of residuals for checking normal distribution
sns.histplot(residual, kde = True)

the graph seems normally distributed.

# Assessing the Linearity Assumption:

The linearity assumption in regression analysis posits a direct, linear relationship between the predictor(s) and the dependent variable. This means that changes in the predictor variables are expected to result in proportional changes in the response variable.

**How to Validate Linearity:**

**Residuals vs. Fitted Values Plot:** One common method to check for linearity is to plot the residuals (the differences between the observed and predicted values) against the predicted (fitted) values. If the assumption of linearity holds, the residuals should display no discernible pattern when plotted against the fitted values. They should be evenly dispersed across the range of fitted values, indicating that the relationship is linear across all values of the predictor variables.


In essence, a lack of clear pattern or structure in this plot suggests that the linearity assumption is reasonable for the data at hand. Any apparent pattern, curve, or systematic structure in the plot would suggest a violation of linearity, indicating that the model may benefit from the inclusion of non-linear terms or a transformation of variables.

In [ ]:
# Predicted values check for linearity
fitted = ols_res_7.fittedvalues

sns.residplot(x = fitted, y = residual, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()


**OBSERVATION:**

The plot of residuals versus fitted values reveals a discernible pattern, indicating that the residuals aren't randomly dispersed. This suggests a potential violation of the linearity assumption.

**Proposed Solution:**

To address this issue, one approach is to apply a log transformation to the target variable. This transformation can often stabilize variance and linearize relationships, making the data more suitable for linear regression modeling. After performing the log transformation, we'll rebuild the model and reassess the residuals to determine if this adjustment leads to an improvement in meeting the linearity assumption.

In [ ]:
# Log transformation on the target variable
import numpy as np
Y_train_log = np.log(Y_train)

# Fitting new model with the transformed target variable
ols_model_7 = sm.OLS(Y_train_log, X_train_scaled_new6)

ols_res_7 = ols_model_7.fit()

# Predicted values
fitted = ols_res_7.fittedvalues

residual1 = ols_res_7.resid

sns.residplot(x = fitted, y = residual1, color = "lightblue")

plt.xlabel("Fitted Values")

plt.ylabel("Residual")

plt.title("Residual PLOT")

plt.show()

**Observations** indicate an absence of discernible patterns in the scatter plot of residuals versus fitted values, suggesting that the linearity assumption is now met. With this encouraging result, the next step is to review the summary statistics of the newly fitted model to evaluate its performance and the significance of the predictors.

In [ ]:
#printing the summary
print(ols_res_7.summary())

Let's check the final assumption of homoscedasticity.

# Evaluating Homoscedasticity:

**Understanding Homoscedasticity:**
Homoscedasticity refers to a condition where the residuals' variance is consistent across all levels of the independent variables. It's a crucial assumption in regression analysis, ensuring that the model's predictive accuracy and confidence intervals remain reliable across the entire range of data.

**Identifying Heteroscedasticity:**
Conversely, heteroscedasticity occurs when the variance of residuals fluctuates across the values of the independent variables. This can manifest as a fan shape or other irregular patterns in the plot of residuals, undermining the reliability of standard errors and test statistics.

**Goldfeld–Quandt Test:**
To formally test for homoscedasticity, we'll employ the Goldfeld-Quandt test. This test compares the variance of residuals in two subsets of the data:

**Null Hypothesis (H0):** The residuals are homoscedastic, meaning the variance is the same across the data.


**Alternative Hypothesis (H1):** The residuals are heteroscedastic, indicating a variance that changes with the independent variables.

By conducting this test, we'll be able to statistically determine whether the variance of the residuals is constant, satisfying the assumption of homoscedasticity, or if it's variable, indicating potential heteroscedasticity that might necessitate further investigation or model adjustments.

In [ ]:
from statsmodels.stats.diagnostic import het_white

from statsmodels.compat import lzip

import statsmodels.stats.api as sms

from statsmodels.compat import lzip

name = ["F statistic", "p-value"]

test = sms.het_goldfeldquandt(Y_train_log, X_train_scaled_new6)

lzip(name, test)

Based on the results of the Goldfeld-Quandt test, the p-value exceeds 0.05, leading us to retain the null hypothesis. This suggests the residuals are homoscedastic, meaning the variance is consistent across the regression line. With all assumptions of the linear regression model now verified, we can confidently proceed with the final model. The derived equation is:

**log(Item_Outlet_Sales)**=4.6356+1.9555⋅**Item_MRP**+0.0158⋅**Item_Fat_Content_Regular**+1.9550⋅**Outlet_Type_SupermarketType1**+1.7737⋅**Outlet_Type_SupermarketType2**+2.4837⋅**Outlet_Type_SupermarketTyp3**

With this model equation established, we're ready to move forward and utilize it for making predictions on the test data.

In [ ]:

# Applying transform on the test data
X_test_scaled = scaler.transform(X_test)

X_test_scaled = pd.DataFrame(X_test_scaled, columns = without_const.columns)

X_test_features = sm.add_constant(X_test_scaled)

X_test_scaled = X_test_scaled.drop(["Item_Weight", "Item_Visibility", "Item_Type_Breads", "Item_Type_Breakfast", "Item_Type_Canned", "Item_Type_Dairy","Item_Type_Frozen Foods","Item_Type_Fruits and Vegetables", "Item_Type_Hard Drinks", "Item_Type_Health and Hygiene", "Item_Type_Household", "Item_Type_Meat", "Item_Type_Others", "Item_Type_Seafood", "Item_Type_Snack Foods", "Item_Type_Soft Drinks", "Item_Type_Starchy Foods", "Outlet_Size_Medium", "Outlet_Size_Small", "Outlet_Location_Type_Tier 2", "Outlet_Location_Type_Tier 3", 'Outlet_Age'], axis = 1)

X_test_scaled.head()

# Evaluation Metrics:

**R-Squared Evaluation:**
The R-squared value serves as a gauge for the model's effectiveness relative to a basic baseline model that doesn't incorporate any independent variables. In this context, our model accounts for approximately 98% of the variance in the data, significantly outperforming the baseline and indicating a high level of explanatory power.

In [ ]:
ols_res_7.rsquared

**Mean Squared Error (MSE) Analysis:**
MSE is a metric that quantifies the average of the squared discrepancies between predicted values and the actual observed values. It essentially provides a mean of the squared errors, offering insight into the overall error magnitude of the model's predictions.

In [ ]:
ols_res_7.mse_resid

**Root Mean Squared Error (RMSE):**
RMSE is an extension of the Mean Squared Error (MSE) metric. By taking the square root of MSE, RMSE is obtained, which scales the error back to the units of the target variable. This adjustment makes the metric more interpretable, providing a clearer understanding of the model's prediction error in relation to the actual values.

In [ ]:
np.sqrt(ols_res_7.mse_resid)

In the following step, we'll assess the model's performance using cross-validation. This technique helps determine whether the model is underfitting, overfitting, or appropriately fitted to the data by evaluating its predictive accuracy across different subsets.

In [ ]:
# Fitting linear model

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

linearregression = LinearRegression()

cv_Score11 = cross_val_score(linearregression, X_train_scaled_new6, Y_train_log, cv = 10)

cv_Score12 = cross_val_score(linearregression, X_train_scaled_new6, Y_train_log, cv = 10,
                             scoring = 'neg_mean_squared_error')


print("RSquared: %0.3f (+/- %0.3f)" % (cv_Score11.mean(), cv_Score11.std()*2))

print("Mean Squared Error: %0.3f (+/- %0.3f)" % (-1*cv_Score12.mean(), cv_Score12.std()*2))

**Observations:**

- The cross-validation R-Squared value is 0.718, closely aligning with the training dataset's R-Squared. This consistency suggests the model is neither overfitting nor underfitting but is rather well-suited to the data.
- Similarly, the Mean Squared Error (MSE) from cross-validation is 0.290, mirroring the training dataset's performance and further supporting the model's appropriate fit.
- The overall evidence indicates that our model is well-calibrated, offering a generalized performance across different data subsets.
- Despite the model's satisfactory performance, it's worth noting that as a linear model, it might not fully capture potential nonlinear patterns in the data. Exploring more advanced regression techniques could potentially unveil and leverage these patterns, potentially enhancing the model's predictive power. However, delving into such complex models falls beyond this case study's scope

In [ ]:
#checking the no. of features in X_test_scaled
X_test_scaled.head()

In [ ]:
#checking the number of features in X_train_Scaled_new6
X_train_scaled_new6.head()

In [ ]:
#As we have observed above there is no const column in X_test_scaled column so we are adding it
from statsmodels.api import add_constant
X_test_scaled_with_const = add_constant(X_test_scaled)
#These predictions will be on log scale
test_predictions = ols_res_7.predict(X_test_scaled_with_const)

In [ ]:
# We are converting the log scale predictions to its original scale
test_predictions_inverse_transformed = np.exp(test_predictions)

test_predictions_inverse_transformed

In [ ]:
#Visulaizing the predictions in both log scale and inverse scale
fig, ax = plt.subplots(1, 2, figsize = (24, 12))

sns.histplot(test_predictions, ax = ax[0]);

sns.histplot(test_predictions_inverse_transformed, ax = ax[1]);

# Conclusion & Recommendations:

Throughout this analysis, we embarked on a comprehensive journey through Exploratory Data Analysis (EDA), examining each variable individually and in relation to others. We addressed missing values with informed strategies based on the relationships between variables.

The modeling process began with a full set of features, from which we methodically eliminated those contributing to multicollinearity and those deemed statistically insignificant. We rigorously tested the linear regression model against its underlying assumptions, refining it iteratively to ensure its validity.

Our evaluation encompassed various metrics, culminating in a robust and interpretable model. The final model equation is as follows:

log(Item_Outlet_Sales)=4.6356+1.9555⋅Item_MRP+0.0158⋅Item_Fat_Content_Regular+∑(Outlet Type Coefficients)log(Item_Outlet_Sales)=4.6356+1.9555⋅Item_MRP+0.0158⋅Item_Fat_Content_Regular+∑(Outlet Type Coefficients)

# Key Interpretations:

**Item_MRP's Impact:** A one-unit increase in Item_MRP is associated with a 1.9555 unit increase in the log of Item_Outlet_Sales. Strategically, positioning higher MRP items in prominent areas could potentially drive sales.

**Supermarket Type 3's Superiority:** Compared to other types, Supermarket Type 3 stores exhibit significantly higher sales. Specifically, their log sales are approximately 1.4 times those of Type 2 and 1.27 times those of Type 1 stores, when other factors are constant.

# Strategic Recommendations:

**Enhance Type 3 Stores:** Given their strong performance, maintaining or further enhancing the appeal of Type 3 supermarkets should be a priority.


**Boost Other Stores' Sales:** For other types, focus on strategies like improving customer service, offering staff training, and increasing the visibility of high MRP items.


**Customized Approaches:** Different strategies may be needed for each store type, considering their unique characteristics and customer base.


In summary, this analysis has not only provided a predictive model but also insights that can inform targeted strategies to optimize sales across different store types.